<a href="https://colab.research.google.com/github/kiannc/BEAD2024/blob/main/colab/workshop1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop 1 - Chua Kian Ann




In [ ]:
# install pyspark using pip
!pip install --ignore-install -q pyspark
# install findspark using pip
!pip install --ignore-install -q findspark

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

# to read in data from a text file, first upload the data file into your google drive and then mount your google drive onto colab
from google.colab import drive
# to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True)
drive.mount('/content/drive', force_remount=True)

In [55]:
driversPath = "/content/drive/MyDrive/data/BEAD_Rebu_Drivers.csv"
taxiCabsPath = "/content/drive/MyDrive/data/BEAD_Rebu_TaxiCabs.json"
passengersPath = "/content/drive/MyDrive/data/BEAD_Rebu_Passengers.csv"
tripDataPath = "/content/drive/MyDrive/data/BEAD_Rebu_TripData.csv"

# the above file is under your pythonProject folder
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()

dfDrivers = spark.read.option("header", "true").option("inferSchema", "true").csv(driversPath)
dfTaxiCabs = spark.read .option("header", "true").option("inferSchema", "true") .option("multiLine", "true").json(taxiCabsPath)
dfPassengers = spark.read.option("header", "true").option("inferSchema", "true").csv(passengersPath)
dfTripData = spark.read.option("header", "true").option("inferSchema", "true").csv(tripDataPath)


In [ ]:
# 1. Retrieve all Driver data (use Drivers.CSV)
dfDrivers.show()

In [ ]:
# 2. Retrieve all Taxis and display the data in ascending order of Taxi License Plate number.

dfTaxiCabs.orderBy(asc("TaxiNumber")).show()


In [ ]:
# 3. Retrieve all Limousine Taxies.
# You should display only the Taxi Number, Taxi Type, and Taxi Colour.
( dfTaxiCabs.select("TaxiNumber", "TaxiType", "TaxiColor")
            .where(col("TaxiType") == "Limosine")
            .show() )

In [ ]:
# 4. Retrieve all 4 seater Premier taxis.

( dfTaxiCabs.where((col("TaxiPassengerCapacity") == 4) & (col("TaxiType") == "Premier"))
            .show() )

In [ ]:
# 5. Determine the average distance per trip based on ALL trips in the month of January 2024.
dfTripData.agg(avg("Distance Travelled")).first()[0]

In [ ]:
# 6. Find the total fares collected grouped by Taxi Type Maxi Cab

# dfTripData.show()
dfTripData.groupBy("Taxi Type").sum("Trip Fare").where(col("Taxi Type") == "Maxi Cab").show()

In [ ]:
# 7. Determine the Average Occupancy i.e., (Number of Passengers / Passenger Capacity) for Standard Taxis
# dfTripData.show()
dfStandardTaxis = dfTripData.where(col("Taxi Type") == "Standard")
dfStandardTaxis.agg(avg("Number Of Passengers")).first()[0]

In [ ]:
# 8. Determine Fares Collected by Day of the Week (ie., Sun, Mon, Tue) for the month of Jan 2024
# dfTripData.show()

dfTripData.where(col("Date").like("%Jan%")).groupBy("Day").sum("Trip Fare").show()

In [ ]:
# 9. Prepare a Tabulation report showing total revenue against
# the two dimensions Hour of the day AND Day of the Week.

dfTripData.rollup("Hour Of Day", "Day").sum("Trip Fare").orderBy("Hour Of Day", "Day").show()

In [ ]:
# 10. Compare the total number of trips per day made by all taxis in weekends vs the total number
# of trips made per day during weekdays in the month of Jan 2024
# dfTripData.show()
# dfTripData.cube("Hour Of Day", "Day").sum("Distance Travelled").orderBy("Hour Of Day", "Day").show()

dfTripDataWeekDays = dfTripData.where(((col("Day") != "Sun") & (col("Day") != "Sat") ) & col("Date").like("%Jan%"))
dfTripDataWeekEnds = dfTripData.where(((col("Day") == "Sun") | (col("Day") == "Sat")) & col("Date").like("%Jan%"))

dfTripDataDistanceTravelled = spark.createDataFrame(
    [
        ("Weekends", dfTripDataWeekDays.agg(sum("Distance Travelled")).first()[0]),
        ("Weekdays", dfTripDataWeekEnds.agg(sum("Distance Travelled")).first()[0]),
    ],
    ["Days", "Distances Travelled"]  # add your column names here
)

dfTripDataDistanceTravelled.show()

In [163]:
# 10. Determine the total fares paid by all Gold Status Passengers in the month of Jan 2024.
# What percentage does this make from the total fares for all customers in month of Jan 2024.

# dfPassengers.show()
# dfTripData.show()
dfPassengersNewHdr = dfPassengers.select(col("PassengerID").alias("Passenger ID"), col("MemSilvererStGoldtus") )

combineDf = dfTripData.join(dfPassengersNewHdr, "Passenger ID")

totalFares = combineDf.agg(sum("Trip Fare"))
totalFaresGold = combineDf.filter(col("MemSilvererStGoldtus") == "Gold").agg(sum("Trip Fare"))

results = (totalFaresGold.collect()[0][0] / totalFares.collect()[0][0]) * 100
print(f"Results: {results}%")

Results: 27.73539443102574%
